In [44]:
import datetime
import json
import random
import boto3
import botocore
# script that writes random data to a DIRECT PUT firehose delivery stream.
# skipping the kinesis data stream part for $$ purposes.
# you lose scalability here.
STREAM_NAME = "jacobs-kinesis-firehose-stream"

def get_data() -> dict:
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)
    }


def generate(stream_name: str, kinesis_client: 'botocore.client.Firehose') -> None:
    # while True: # this makes an infinite loop to continuously send messages
    data = get_data()
    print(data)
    kinesis_client.put_record(
        DeliveryStreamName=stream_name,
        Record={
            'Data': json.dumps(data) + '\n'
        },
    )
        # PartitionKey="partitionkey")


# if __name__ == '__main__':
#     generate(STREAM_NAME, boto3.client('kinesis'))

In [55]:
generate(STREAM_NAME, boto3.client('firehose'))

{'EVENT_TIME': '2022-07-27T14:32:07.580844', 'TICKER': 'INTC', 'PRICE': 99.65}


In [ ]:
# data gets loaded in like this without using '\n' at end of json msg.
{"EVENT_TIME": "2022-07-27T13:40:19.345362", "TICKER": "TBV", "PRICE": 27.96}{"EVENT_TIME": "2022-07-27T13:40:19.467695", "TICKER": "AAPL", "PRICE": 37.42}{"EVENT_TIME": "2022-07-27T13:40:19.502649", "TICKER": "MSFT", "PRICE": 73.61}{"EVENT_TIME": "2022-07-27T13:40:19.569286", "TICKER": "AAPL", "PRICE": 55.77}{"EVENT_TIME": "2022-07-27T13:40:19.604135", "TICKER": "INTC", "PRICE": 59.82}{"EVENT_TIME": "2022-07-27T13:40:19.640605", "TICKER": "AAPL", "PRICE": 16.61}


# so after adding the \n it turns into this
{"EVENT_TIME": "2022-07-27T13:40:19.345362", "TICKER": "TBV", "PRICE": 27.96}
{"EVENT_TIME": "2022-07-27T13:40:19.467695", "TICKER": "AAPL", "PRICE": 37.42}
{"EVENT_TIME": "2022-07-27T13:40:19.502649", "TICKER": "MSFT", "PRICE": 73.61}
{"EVENT_TIME": "2022-07-27T13:40:19.569286", "TICKER": "AAPL", "PRICE": 55.77}
{"EVENT_TIME": "2022-07-27T13:40:19.604135", "TICKER": "INTC", "PRICE": 59.82}
{"EVENT_TIME": "2022-07-27T13:40:19.640605", "TICKER": "AAPL", "PRICE": 16.61}

# the other thing you can do is a use a lambda function to process the data before every batch gets written to s3, so you can separate the json records there.
# but ,,,, fk that